In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import random
import os

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [2]:
search = '가상자산'

In [4]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles_temp = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
                    new_urls.append(article_links[idx + 1])
                    idx += 2
                else:
                    new_urls.append(article_links[idx])
                    idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        if idx == len(article_links) - 1:
            new_urls.append(article_links[idx])

        # 각 페이지에서 URL이 10개가 아닌 경우 해당 페이지의 URL 출력
        if len(new_urls) % 10 != 0:
            print(f"Warning: {url} contains {len(new_urls)} links instead of 10.")
        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []
        current_titles = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press_name = article_infos[idx - 1]
                    
                    if "CHIEF EXECUTIVE" not in current_press_name:
                        current_press.append(current_press_name)
                        current_titles.append(article_titles_temp[len(current_dates)])
                        current_dates.append(match.group(0))

        articles.extend(new_urls[:len(current_titles)])  # articles length should match with titles
        titles.extend(current_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(random.uniform(1, 1.5))
        
    return articles, titles, dates, press

In [5]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    
    for url in tqdm(article_urls):
        if 'news.naver.com' in url:
            news = requests.get(url, headers=headers)
            soup = BeautifulSoup(news.text, "html.parser")

            content = soup.select("#dic_area") or soup.select("#articeBody")
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
            contents.append(cleaned_content)

            try:
                date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
            except AttributeError:
                date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
            article_time.append(date)  # 이름 변경
        else:
            contents.append('')
            article_time.append('')
    
    return contents, article_time

In [6]:
# def main():
#     search = input("검색할 키워드를 입력해주세요:")
#     start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
#     end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

#     urls = makeUrl(search, start_date, end_date)
#     article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
#     contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
#     df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
#     df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#     return df

# if __name__ == "__main__":
#     result_df = main()
#     print(result_df)

In [33]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=369):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [34]:
# search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))
print("Length of contents:", len(contents))
print("Length of article_time:", len(article_time))

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls }) 
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")

100%|██████████| 3690/3690 [08:00<00:00,  7.68it/s]

Length of article_urls: 3690
Length of article_titles: 3690
Length of article_dates: 3690
Length of article_press: 3690
Length of contents: 3690
Length of article_time: 3690
            date                 time  \
0     2023.08.09  2023-08-09 00:01:10   
1     2023.08.09                        
2     2023.08.09                        
3     2023.08.09                        
4     2023.08.09                        
...          ...                  ...   
3668  2023.08.30                        
3669  2023.08.30  2023-08-30 11:44:03   
3670  2023.08.30                        
3671  2023.08.30                        
3672  2023.08.30  2023-08-30 11:48:01   

                                                  title  \
0                 [사설] 혁신위로 당내 분란 커지는데 이 대표는 손 놓고 있을 건가   
1     [속보] 미국 CPI 물가 예상밖 "급등" 뉴욕증시 비트코인 국채금리 "2차 발작"...   
2                        페이팔 스테이블코인 출시···가상자산 결제 대중화 성큼   
3         뉴욕증시 은행주 대폭락 "무디스 신용등급 강등 일파만파" … 국채금리 달러환...   
4                             델리오, 11일

In [35]:
df

,date,time,title,content,press,link
0,2023.08.09,2023-08-09 00:01:10,[사설] 혁신위로 당내 분란 커지는데 이 대표는 손 놓고 있을 건가,\n내일 ‘대의원제 축소’ 쇄신안 발표비명 “강성 지지층 입김 커져” 반발책임 통감...,A27면 1단,https://n.news.naver.com/mnews/article/022/000...
1,2023.08.09,,"[속보] 미국 CPI 물가 예상밖 ""급등"" 뉴욕증시 비트코인 국채금리 ""2차 발작""...",,글로벌이코노믹,http://www.g-enews.com/
2,2023.08.09,,페이팔 스테이블코인 출시···가상자산 결제 대중화 성큼,,디센터,http://decenter.kr
3,2023.08.09,,"뉴욕증시 은행주 대폭락 ""무디스 신용등급 강등 일파만파"" … 국채금리 달러환...",,글로벌이코노믹,http://www.g-enews.com/
4,2023.08.09,,"델리오, 11일부터 웹 ·앱 서비스 중단 예고",,디지털투데이,http://www.digitaltoday.co.kr/
...,...,...,...,...,...,...
3668,2023.08.30,,상반기 금융사 '의심거래 보고' 증가…증권사 증가율 '톱',,신아일보,http://www.shinailbo.co.kr/
3669,2023.08.30,2023-08-30 11:44:03,"[뉴스포커스] 윤대통령, 연일 '이념' 강조…'홍범도 흉상' 두고 논쟁 가열",\n\t\t\t&lt;출연 : 최수영 시사평론가·김성완 시사평론가&gt;윤석열 대통...,연합뉴스TV,https://n.news.naver.com/mnews/article/422/000...
3670,2023.08.30,,커지는 자금세탁 우려…금융사 '의심 거래(STR)' 증가,,금융소비자뉴스,http://newsfc.co.kr/
3671,2023.08.30,,민관 함께 2조 규모 '스타트업 코리아 펀드' 조성…한국인 해외 창업도 지원한...,,이투데이,http://www.etoday.co.kr


In [36]:
# 데이터 프레임 저장
path = r"C:\\Users\\boyu571\\boyu571_Github\\01_Kakaobank_SKKU_Research_23\\data_gatherting"
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
full_path = os.path.join(path, filename)
df.to_csv(full_path, encoding='utf-8-sig', index=False)

In [74]:
# result = pd.concat([df1, df2, df3, df4], ignore_index=True)
# result.drop_duplicates(keep='first', inplace=True, ignore_index=True)
# result

,date,time,title,content,press,link
0,2021.02.01,2021-02-01 00:00:19,'극한의 롤러코스터'…게임스탑發 증시 폭락 우려 확산,\n'극한의 롤러코스터' 게임스탑 격변의 한주한주간 400% 폭등…AMC 등도 주가...,이데일리언론사 선정,https://n.news.naver.com/mnews/article/018/000...
1,2021.02.01,,"뉴욕증시 비트코인 거품 붕괴 조짐, 게임스톱 공매도 일파만파… 테슬라 애플...",,글로벌이코노믹,http://www.g-enews.com/
2,2021.02.01,,"'애로부부' 현영, 결혼생활 10년에도 남편과 뜨거운 이유? ""50살에 다른 매력...",,뉴스웍스,http://www.newsworks.co.kr/
3,2021.02.01,2021-02-01 02:01:33,마디지수 깨진 글로벌 증시…유동성 랠리 끝나나,"\n\t\t\t[이데일리 권소현 기자, 뉴욕=김정남 특파원] 사상 최고치 경신을 거...",A1면 TOP,https://n.news.naver.com/mnews/article/018/000...
4,2021.02.01,2021-02-01 03:03:03,[밀레니얼 톡] 당당하게 ‘돈 이야기’ 하며 삽시다,\n\t\t\t\t\t\t\t\t\t\t90년대생 세 명을 인터뷰했다. 주제는 ‘주...,A29면 1단,https://n.news.naver.com/mnews/article/023/000...
...,...,...,...,...,...,...
12658,2021.03.31,,인도가 '비트코인 금지법'을 도입하면?,,코인데스크코리아,https://www.coindeskkorea.com/
12659,2021.03.31,,[업비트 특징주]비트토렌트 코인 주가 10% 이상 상승,,국제뉴스,http://www.gukjenews.com/
12660,2021.03.31,,트리핀의 딜레마,,단비뉴스,http://www.danbinews.com
12661,2021.03.31,,NFT,,단비뉴스,http://www.danbinews.com


In [75]:
# result.to_csv('비트코인_20210201_20210331.csv', encoding='utf-8-sig', index=False)